In [1]:
from numpy import array

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [10]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
n_features = 1

In [4]:
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [14]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [15]:
X

array([[[10],
        [20],
        [30]],

       [[20],
        [30],
        [40]],

       [[30],
        [40],
        [50]],

       [[40],
        [50],
        [60]],

       [[50],
        [60],
        [70]],

       [[60],
        [70],
        [80]]])

In [8]:
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout  

In [16]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [17]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [19]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
yhat

array([[102.35276]], dtype=float32)

In [21]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import itertools
import warnings
from statsmodels.tsa.stattools import acf, pacf
import math
import datetime

In [22]:
mydata = pd.read_csv("train.csv")
subset = mydata.loc[mydata.building_number == 1]
subset = subset[['timestamp', 'main_meter']]
subset['timestamp'] = pd.to_datetime(subset['timestamp'], format = '%Y-%m-%d %H:%M:%S')
subset = subset.sort_values(by=['timestamp'])
subset = subset.loc[subset.timestamp >= '2017-09-01 00:00:00']
subset = subset.set_index('timestamp')
subset = subset[['main_meter']]

In [23]:
train = subset.iloc[0:11616]
test = subset.iloc[11616:11712]

In [25]:
from numpy import array

In [26]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [27]:
# choose a number of time steps
n_steps = 3
n_features = 1

In [29]:
# split into samples
X, y = split_sequence(train.main_meter, n_steps)

In [32]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [34]:
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout  

In [35]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [36]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [51]:
# split into samples
x_input, y_input = split_sequence(test.main_meter, n_steps)

In [52]:
# demonstrate prediction
#x_input = array([70, 80, 90])
x_input = x_input.reshape((93, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
yhat

array([[1736.7787],
       [1560.7094],
       [2000.1895],
       [1868.7953],
       [1977.926 ],
       [1936.4457],
       [1785.2474],
       [2139.7136],
       [1957.5637],
       [2028.8138],
       [1749.296 ],
       [1893.764 ],
       [2096.03  ],
       [1858.7126],
       [1806.9694],
       [1875.8527],
       [1725.4036],
       [1752.0742],
       [1609.348 ],
       [1869.9913],
       [1990.1342],
       [3266.5242],
       [6699.337 ],
       [7399.075 ],
       [6411.641 ],
       [5629.6475],
       [7137.8647],
       [6859.93  ],
       [6870.483 ],
       [5457.794 ],
       [6953.2456],
       [6313.142 ],
       [6554.4727],
       [6258.5093],
       [4716.9526],
       [6827.6245],
       [6439.4204],
       [6658.941 ],
       [6793.093 ],
       [6925.4556],
       [6081.716 ],
       [6367.8706],
       [6679.767 ],
       [6394.813 ],
       [6573.215 ],
       [6424.895 ],
       [6772.517 ],
       [8142.0767],
       [7386.436 ],
       [6963.705 ],


In [75]:
ypred = []
for i in yhat:
    ypred.extend(i)
ypred = np.array(ypred)

In [76]:
RMSE = math.sqrt(sum((ypred-np.array(test.main_meter)[3:96])**2))
RMSE

6385.8462453669545

In [77]:
MAPE = 100*(sum(abs(ypred - np.array(test.main_meter)[3:96])/np.array(test.main_meter)[3:96])/93)
MAPE

9.037647201967463

In [78]:
for i in range(len(ypred)):
    print(np.array(test.main_meter)[3:96][i], ypred[i])

1536.2864085776334 1736.7787
1887.0632459250353 1560.7094
1922.8414546482254 2000.1895
1964.9757536290272 1868.7953
1935.5101372438996 1977.926
1785.1132749289786 1936.4457
2053.4657132650314 1785.2474
2008.529434430884 2139.7136
2006.5113154974906 1957.5637
1768.4366203995214 2028.8138
1820.5251635544712 1749.296
2057.7318164181615 1893.764
1916.378981769032 2096.03
1780.6911967907834 1858.7126
1823.5632687917582 1806.9694
1730.5078555516761 1875.8527
1715.7306780090312 1725.4036
1602.7833109760172 1752.0742
1798.0543562284695 1609.348
1988.220314968283 1869.9913
3136.0019083186144 1990.1342
6345.435040821334 3266.5242
7025.887233344343 6699.337
6689.7873344187265 7399.075
5765.947009060977 6411.641
6870.231858845332 5629.6475
7141.575161736657 7137.8647
7032.9394410724035 6859.93
5704.712473036167 6870.483
6698.8403216320885 5457.794
6616.48382336527 6953.2456
6616.441395956834 6313.142
6371.529401343065 6554.4727
4964.785967985775 6258.5093
6532.645373184357 4716.9526
6646.785460638